In [1]:
#!pip install mysql-connector-python
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# Define your MySQL connection parameters
db_config = {
    'user': 'root',
    'password': 'Mithun@23',
    'host': '127.0.0.1',  
    'database': 'football'
}

# Connect to the database
conn = mysql.connector.connect(**db_config)

In [3]:
#Define queries
queries = {
    'players': 'SELECT * FROM players',
    'games': 'SELECT * FROM games',
    'appearance': 'SELECT * FROM appearances',
    'lineup': 'SELECT * FROM game_lineups',
    'events': 'SELECT * FROM game_events'
}

# Create a dictionary to store DataFrames
dfs = {}

# Fetch data from each table and store it in a DataFrame
for table, query in queries.items():
    dfs[table] = pd.read_sql(query, conn)

C:\Users\User\AppData\Local\Temp\ipykernel_352\2037194353.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[table] = pd.read_sql(query, conn)


In [5]:
#conn.close()


In [128]:
import numpy as np
# Replace blank values and spaces with NaN in each dataframe in the dictionary 'dfs'
for table, df in dfs.items():
    df.replace('', np.nan, inplace=True)
    df.replace(' ', np.nan, inplace=True)  
    print(f"Blank values in table: {table}")
    print(df.isna().sum(), "\n")

Blank values in table: players
ï»¿player_id                    0
name                            0
last_season                     0
current_club_id                 0
player_code                     0
country_of_birth                0
date_of_birth                   0
sub_position                    0
position                        0
foot                           11
height_in_cm                    3
market_value_in_eur            38
highest_market_value_in_eur     4
contract_expiration_date       46
agent_name                     45
dtype: int64 

Blank values in table: games
ï»¿game_id                   0
competition_id               0
season                       0
round                        0
date                         0
home_club_goals              0
away_club_goals              0
home_club_position         479
away_club_position         479
home_club_manager_name       5
away_club_manager_name       5
stadium                      0
attendance                 115
referee     

DATA MERGING 

In [129]:
# Rename columns to remove non-ASCII characters in column names
dfs['games'] = dfs['games'].rename(columns={'ï»¿game_id': 'game_id'})
dfs['players'] = dfs['players'].rename(columns={'ï»¿player_id': 'player_id'})
dfs['appearance'] = dfs['appearance'].rename(columns={'ï»¿appearance_id': 'appearance_id'})
dfs['lineup'] = dfs['lineup'].rename(columns={'ï»¿game_lineups_id': 'game_lineups_id'})
dfs['events'] = dfs['events'].rename(columns={'ï»¿game_event_id': 'game_event_id'})

# Merge 'games' and 'appearance' dataframes on common columns
common_columns = list(set(dfs['games'].columns) & set(dfs['appearance'].columns))
merged_df_1 = dfs['games'].merge(dfs['appearance'], on=common_columns, how='left')

# Merge the resulting dataframe with 'players' dataframe on common columns
common_columns = list(set(merged_df_1.columns) & set(dfs['players'].columns))
merged_df_2 = merged_df_1.merge(dfs['players'], on=common_columns, how='left')

# Merge the resulting dataframe with 'events' dataframe on common columns
common_columns = list(set(merged_df_2.columns) & set(dfs['events'].columns))
merged_df_3 = merged_df_2.merge(dfs['events'], on=common_columns, how='left')

# Merge the resulting dataframe with 'lineup' dataframe on common columns
common_columns = list(set(merged_df_3.columns) & set(dfs['lineup'].columns))
final = merged_df_3.merge(dfs['lineup'], on=common_columns, how='left')
final

game_id competition_id  season         round        date  \
0     2222734            RU1    2012  19. Matchday  07-12-2012   
1     2224572            DK1    2012   3. Matchday  28-07-2012   
2     2224628            DK1    2012  22. Matchday  08-03-2013   
3     2224655            DK1    2012  22. Matchday  10-03-2013   
4     2224729            DK1    2012   1. Matchday  15-07-2012   
...       ...            ...     ...           ...         ...   
3717  2875216            DK1    2017  19. Matchday  10-12-2017   
3718  2875216            DK1    2017  19. Matchday  10-12-2017   
3719  3099247            BE1    2018  11. Matchday  20-10-2018   
3720  2872273            GB1    2017  16. Matchday  09-12-2017   
3721  2517322            FAC    2014   Third Round  04-01-2015   

      home_club_goals  away_club_goals  home_club_position  \
0                   2                3                11.0   
1                   1                2                12.0   
2                   2                0                 3.0   
3                   0                3                 6.0   
4                   0                1                 9.0   
...               ...              ...                 ...   
3717                3                2                 3.0   
3718                3                2                 3.0   
3719                3                1                 9.0   
3720                5                1                 6.0   
3721                3                1                 NaN   

      away_club_position home_club_manager_name  ...         agent_name  \
0                   10.0        Miodrag Bozovic  ...           Prosport   
1                    1.0            Jess Thorup  ...               None   
2                   10.0             Colin Todd  ...               None   
3                   12.0           Peter Rensen  ...               None   
4                    5.0        Auri Skarbalius  ...               None   
...                  ...                    ...  ...                ...   
3717                 4.0        Kasper Hjulmand  ...  Avid Sports Group   
3718                 4.0        Kasper Hjulmand  ...               None   
3719                 7.0          Glen De Boeck  ...        CAA Stellar   
3720                15.0    Mauricio Pochettino  ...         CMG Sports   
3721                 NaN            Mark Hughes  ...         CMG Sports   

                         game_event_id  minute           type description  \
0                                  NaN     NaN            NaN         NaN   
1                                  NaN     NaN            NaN         NaN   
2                                  NaN     NaN            NaN         NaN   
3                                  NaN     NaN            NaN         NaN   
4                                  NaN     NaN            NaN         NaN   
...                                ...     ...            ...         ...   
3717                               NaN     NaN            NaN         NaN   
3718  237567d50f0a3d2c545dc4776a8321f6    84.0  Substitutions        None   
3719  d3e7898d2f3e2ee405a0caa5e23ad4fb    30.0  Substitutions        None   
3720                               NaN     NaN            NaN         NaN   
3721  ab76dcda1ea48a36f4bc2b990a7dced2    76.0  Substitutions        None   

     player_in_id player_assist_id game_lineups_id number team_captain  
0             NaN              NaN             NaN    NaN          NaN  
1             NaN              NaN             NaN    NaN          NaN  
2             NaN              NaN             NaN    NaN          NaN  
3             NaN              NaN             NaN    NaN          NaN  
4             NaN              NaN             NaN    NaN          NaN  
...           ...              ...             ...    ...          ...  
3717          NaN              NaN             NaN    NaN          NaN  
3718       148343             None             NaN    NaN 

In [130]:
final.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date',
       'home_club_goals', 'away_club_goals', 'home_club_position',
       'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee',
       'home_club_formation', 'away_club_formation', 'home_club_name',
       'away_club_name', 'aggregate', 'competition_type', 'appearance_id',
       'player_id', 'player_name', 'yellow_cards', 'red_cards', 'goals',
       'assists', 'minutes_played', 'name', 'last_season', 'current_club_id',
       'player_code', 'country_of_birth', 'date_of_birth', 'sub_position',
       'position', 'foot', 'height_in_cm', 'market_value_in_eur',
       'highest_market_value_in_eur', 'contract_expiration_date', 'agent_name',
       'game_event_id', 'minute', 'type', 'description', 'player_in_id',
       'player_assist_id', 'game_lineups_id', 'number', 'team_captain'],
      dtype='object')

DATA CLEANING 

In [131]:
# # Define the columns to keep based on the focus areas
# columns_to_keep = [
#     'home_club_goals',
#     'away_club_goals',
#     'season',
#     'date',
#     'player_name',
#     'goals',
#     'assists',
#     'position',
#     'market_value_in_eur',
#     'height_in_cm',
#     'date_of_birth',
#     'home_club_name',
#     'away_club_name',
#     'home_club_position',
#     'away_club_position',
#     'stadium',
#     'attendance',
#     'referee',
#     'yellow_cards',
#     'red_cards',
#     'minute',
#     'type',
#     'competition_type',
#     'contract_expiration_date'
# ]

# # Filter the DataFrame to keep only the relevant columns
# final = final[columns_to_keep]
# final

In [132]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3722 entries, 0 to 3721
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   game_id                      3722 non-null   int64  
 1   competition_id               3722 non-null   object 
 2   season                       3722 non-null   int64  
 3   round                        3722 non-null   object 
 4   date                         3722 non-null   object 
 5   home_club_goals              3722 non-null   int64  
 6   away_club_goals              3722 non-null   int64  
 7   home_club_position           3180 non-null   float64
 8   away_club_position           3180 non-null   float64
 9   home_club_manager_name       3716 non-null   object 
 10  away_club_manager_name       3716 non-null   object 
 11  stadium                      3722 non-null   object 
 12  attendance                   3586 non-null   float64
 13  referee           

In [133]:
#Missing values
final.isna().sum()

game_id                           0
competition_id                    0
season                            0
round                             0
date                              0
home_club_goals                   0
away_club_goals                   0
home_club_position              542
away_club_position              542
home_club_manager_name            6
away_club_manager_name            6
stadium                           0
attendance                      136
referee                           4
home_club_formation            3722
away_club_formation            3722
home_club_name                  126
away_club_name                   66
aggregate                         0
competition_type                  0
appearance_id                     0
player_id                         0
player_name                       0
yellow_cards                      0
red_cards                         0
goals                             0
assists                           0
minutes_played              

In [134]:
#Remove columns that have no values at all
final.drop(['home_club_formation','away_club_formation','game_lineups_id','number','team_captain'],axis = 1,inplace = True)

In [135]:
final['contract_expiration_date'].value_counts()

30-06-2024 00:00    529
31-12-2023 00:00    471
30-06-2025 00:00    413
31-12-2024 00:00    330
30-06-2027 00:00    258
31-12-2025 00:00    198
30-06-2028 00:00     60
30-06-2026 00:00     41
30-11-2023 00:00     23
31-05-2026 00:00      6
31-12-2026 00:00      1
Name: contract_expiration_date, dtype: int64

In [136]:
final['referee'].value_counts()

Dr. Felix Brych       75
Felix Zwayer          74
Marco Fritz           71
Manuel Graofe         66
Deniz Aytekin         64
                      ..
Kristinn Jakobsson     1
Alper Ulusoy           1
Manuel Oliveira        1
Cosme Machado          1
Antti Munukka          1
Name: referee, Length: 347, dtype: int64

In [139]:
final['foot'].value_counts()

right    2954
left      480
both      197
Name: foot, dtype: int64

In [140]:
final['agent_name'].value_counts()

Wasserman               541
YMU Group               304
CAA Stellar             227
Robert Schneider        207
ROGON                   178
CMG Sports              175
Mega Sports             128
YMU Management Ltd.      97
PRO FC                   93
Unique Sports Group      86
Promoesport              78
Avid Sports Group        66
Gestifute                66
Football Company Srl     47
PROSPORT Management      47
SBM                      41
OmniSports               40
Joes Blakborn            39
ARP Sportmarketing       37
BS Group - BS Law        28
in4 sports               27
Prosport                 22
acta7                    10
Octagon                   9
TrueSports GmbH           8
FC Enterprise             8
TOP Agency                7
GROW                      5
PG 120 Sport Agency       4
BR Group Management       3
CCC                       3
athleteMNGment            2
Field Management          2
CAA Base Ltd              2
NVA SEG                   1
AKA Global GmbH     

In [142]:
final['player_in_id'].value_counts()

41511     11
90466      9
32711      9
392768     7
102226     6
          ..
122011     1
57372      1
76077      1
99331      1
34838      1
Name: player_in_id, Length: 380, dtype: int64

In [141]:
final['description'].value_counts()

, Tactical                                                                             78
, Not reported                                                                         64
1. Yellow card  , Foul                                                                 28
, Right-footed shot, 1. Goal of the Season                                             25
2. Yellow card  , Foul                                                                 20
                                                                                       ..
, Left-footed shot, 14. Goal of the Season Assist:                                      1
, Left-footed shot, 9. Goal of the Season Assist: , Pass, 3. Assist of the Season       1
, Long distance kick, 5. Goal of the Season Assist:                                     1
8. Yellow card  , Foul                                                                  1
, Long distance kick, 7. Goal of the Season Assist: , Pass, 3. Assist of the Season     1
Name: desc

In [144]:
final['home_club_manager_name'].value_counts()

Mark Hughes          74
Paul Lambert         64
Roberto MartÃ­nez    64
Lucien Favre         61
Dieter Hecking       59
                     ..
Patrick Vieira        1
Yuri Semin            1
Bernard Blaquart      1
Stephane Jobard       1
Luis Freire           1
Name: home_club_manager_name, Length: 580, dtype: int64

In [143]:
final['player_assist_id'].value_counts()

36391     7
122421    6
9602      6
25427     6
39381     5
         ..
125250    1
77001     1
15243     1
69646     1
3785      1
Name: player_assist_id, Length: 64, dtype: int64

In [145]:
final['away_club_manager_name'].value_counts()

Mark Hughes          80
Paul Lambert         73
Lucien Favre         69
Roberto MartÃ­nez    63
Dieter Hecking       59
                     ..
Ian Murray            1
Steve Lovell          1
Raul Procopio         1
Orest Lenczyk         1
Kevin Wilkin          1
Name: away_club_manager_name, Length: 550, dtype: int64

In [11]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Columns with numerical missing values
# Impute missing values

# List of numerical columns to be imputed
num_cols = [
    'market_value_in_eur', 
    'height_in_cm', 
    'home_club_position', 
    'away_club_position', 
    'attendance', 
    'minute'
]

# Imputer for numerical columns with median strategy
imputer_num = SimpleImputer(strategy='median')
final[num_cols] = imputer_num.fit_transform(final[num_cols])

# List of categorical columns to be imputed
cat_cols = [
    'referee', 
    'type',
    'home_club_name',
    'away_club_name',
    'home_club_manager_name',
    'away_club_manager_name',
    'foot',
    'contract_expiration_date',
    'agent_name',
    'game_event_id',
    'description',
    'player_in_id',
    'player_assist_id'
]

# Imputer for categorical columns with most frequent strategy
imputer_cat = SimpleImputer(strategy='most_frequent')
final[cat_cols] = imputer_cat.fit_transform(final[cat_cols])

# Replace specific missing values with default values
# Replace missing values in 'contract_expiration_date' with a default date
final['contract_expiration_date'].replace(pd.NaT, '30-06-2024 00:00', inplace=True)

# Replace missing values in 'referee' with a default referee name
final['referee'].replace(pd.NaT, 'Dr. Felix Brych', inplace=True)

# Replace missing values in 'home_club_name' and 'away_club_name' with 'Borussia Dortmund'
final['home_club_name'].replace(pd.NaT, 'Borussia Dortmund', inplace=True)
final['away_club_name'].replace(pd.NaT, 'Borussia Dortmund', inplace=True)

# Replace missing values in 'foot' with 'right'
final['foot'].replace(pd.NaT, 'right', inplace=True)

# Replace missing values in 'description' with ', Tactical'
final['description'].replace(pd.NaT, ', Tactical', inplace=True)

# Replace missing values in 'player_in_id' with '41511'
final['player_in_id'].replace(pd.NaT, '41511', inplace=True)

# Replace missing values in 'player_assist_id' with '36391'
final['player_assist_id'].replace(pd.NaT, '36391', inplace=True)

# Replace missing values in 'agent_name' with 'Wasserman'
final['agent_name'].replace(pd.NaT, 'Wasserman', inplace=True)

# Replace missing values in 'home_club_manager_name' and 'away_club_manager_name' with 'Mark Hughes'
final['home_club_manager_name'].replace(pd.NaT, 'Mark Hughes', inplace=True)
final['away_club_manager_name'].replace(pd.NaT, 'Mark Hughes', inplace=True)

# Print the final dataframe to verify the imputations
print(final.head())


   game_id competition_id  season         round       date  home_club_goals  \
0  2222734            RU1    2012  19. Matchday  12/7/2012                2   
1  2222734            RU1    2012  19. Matchday  12/7/2012                2   
2  2224572            DK1    2012   3. Matchday  7/28/2012                1   
3  2224572            DK1    2012   3. Matchday  7/28/2012                1   
4  2224628            DK1    2012  22. Matchday   3/8/2013                2   

   away_club_goals  home_club_position  away_club_position  \
0                3                11.0                10.0   
1                3                11.0                10.0   
2                2                12.0                 1.0   
3                2                12.0                 1.0   
4                0                 3.0                10.0   

  home_club_manager_name  ... agent_name                     game_event_id  \
0        Miodrag Bozovic  ...  Wasserman  0010a0fa5d07cb56507dafd5b6c9b9ca

In [147]:
final.isna().sum()

game_id                        0
competition_id                 0
season                         0
round                          0
date                           0
home_club_goals                0
away_club_goals                0
home_club_position             0
away_club_position             0
home_club_manager_name         0
away_club_manager_name         0
stadium                        0
attendance                     0
referee                        0
home_club_name                 0
away_club_name                 0
aggregate                      0
competition_type               0
appearance_id                  0
player_id                      0
player_name                    0
yellow_cards                   0
red_cards                      0
goals                          0
assists                        0
minutes_played                 0
name                           0
last_season                    0
current_club_id                0
player_code                    0
country_of

In [148]:
final.to_csv("Football_data_merged_cleaned.csv")

In [149]:
final.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date',
       'home_club_goals', 'away_club_goals', 'home_club_position',
       'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee',
       'home_club_name', 'away_club_name', 'aggregate', 'competition_type',
       'appearance_id', 'player_id', 'player_name', 'yellow_cards',
       'red_cards', 'goals', 'assists', 'minutes_played', 'name',
       'last_season', 'current_club_id', 'player_code', 'country_of_birth',
       'date_of_birth', 'sub_position', 'position', 'foot', 'height_in_cm',
       'market_value_in_eur', 'highest_market_value_in_eur',
       'contract_expiration_date', 'agent_name', 'game_event_id', 'minute',
       'type', 'description', 'player_in_id', 'player_assist_id'],
      dtype='object')